In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display, Image

In [2]:
'''
s = source
t = target
g = grass
f = forest
h = hill
m = mountain
w = wall
'''

map_1 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','g','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_2 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','f','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_3 = [['s','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','t'],
        ]

maps = [map_1, map_2, map_3]
map_files = ['small_map1.png', 'small_map2.png', 'small_map3.png']

directions = [[-1, 0 ], # Up
             [ 0, -1],  # Left
             [ 1, 0 ],  # Down
             [ 0, 1 ]]  # Right

direction_symbols = ["^", # Up 
                     "<", # Left
                     "V", # Down
                     ">"] # Right

def print_grid(name, grid):
    print(name + ": ")
    for row in grid:
        print(row)
    print()

### Let's Do This!
For the remainder of this module we are going to use graphs that are represented by grids. This will help us visualize pathfinding a bit easier. It also makes the code more straightforward. We will be storing these maps in 2D Python lists that store different terrain types as _chars_. 

```python
'''
s = source
t = target
g = grass
f = forest
h = hill
m = mountain
w = wall
'''

map_1 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','g','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

``` 
<img src="./images/small_map1.png" align="left"/>
<div style="clear: both;"></div>

So now that we have a way to represent our graphs as a map we need a way to convert this into something usable by our algoirthm. So let's make a Graph _class_! What does this class need to do?

1. Parse map of chars into cost of movement into each grid square
2. Know map specifics such as height, width, source, and target
3. Provide access to this information

```python
# Accepts a map of chars as grid
def parse_map(self, grid):
        # Save dimensions
        self.height = len(grid)
        self.width = len(grid[0])
        
        # Set source and target in case the map doesn't specify
        self.source = [0,0]
        self.target = [len(grid)-1, len(grid[0])-1]
    
        # Build an array of same dimensions to store costs of movement (default 1)
        self.costs = [[1 for col in range(len(grid[row]))] for row in range(len(grid))]
    
        # Iterate over map of chars
        for row in range(len(self.costs)):
            for col in range(len(self.costs[0])):
                # Found the source!
                if grid[row][col] == 's':
                    self.source = [row,col]
                # Found the target!
                elif grid[row][col] =='t':
                    self.target = [row,col]
                # Look up the cost for terrain type and assign to costs grid
                else:
                    self.costs[row][col] = self.cost_legend[grid[row][col]]
```

The rest of the functionality is straightforward so see the complete _Graph_ class below.

In [9]:
class Graph:
    
    # Cost for moving into each terrain type
    cost_legend = {'g':1, 'f':2, 'h':3, 'm':4, 'w':float('inf')}
    
    def __init__(self, grid):
        self.map = grid
        self.parse_map(grid)
        
    # Accepts a map of chars as grid
    def parse_map(self, grid):
        # Save dimensions
        self.height = len(grid)
        self.width = len(grid[0])
        
        # Set source and target in case the map doesn't specify
        self.source = [0,0]
        self.target = [len(grid)-1, len(grid[0])-1]
    
        # Build an array of same dimensions to store costs of movement (default 1)
        self.costs = [[1 for col in range(len(grid[row]))] for row in range(len(grid))]
    
        # Iterate over map of chars
        for row in range(len(self.costs)):
            for col in range(len(self.costs[0])):
                # Found the source!
                if grid[row][col] == 's':
                    self.source = [row,col]
                # Found the target!
                elif grid[row][col] =='t':
                    self.target = [row,col]
                # Look up the cost for terrain type and assign to costs grid
                else:
                    self.costs[row][col] = self.cost_legend[grid[row][col]]
       
    def in_bounds(self, loc):
        row, col = loc
        return 0 <= row < self.height and 0 <= col < self.width
    
    def cost(self, loc):
        row, col = loc
        return self.costs[row][col]

### Priority Queue
Now that we can represent our map as a graph, we need to have a way for Dijkstra's algorithm to keep track of our ever expanding list of nodes that need processing. For this we need a _Priority Queue_. While there are ready-made classes available we don't need all those features and they are easy to implement. For our purposes the _Priority Queue_ needs the following features:

1. Store all the elements
2. Add elements to queue
3. Sort elements so the item with the highest priority is at the front
4. Remove the first item and return it

For our implementation we will use a simple List which we will call _Q_ (get it?). When adding elements to the queue we will simply append them to the list. To ensure that the highest priority element is on the top we will use the built in list _sort()_ function. This sorts from highest to lowest priority so when using retrieving the highest priority element we need to use _pop(0)_. The _empty()_ function will just check the length of the queue.

Please see the full class below.

In [48]:
class Priority_Queue():
    
    def __init__(self):
        self.Q = []
        
    def put(self, item):
        self.Q.append(item)
        self.Q.sort()
        
    def get(self):
        if not self.empty():
            item = self.Q.pop(0)
            return item
        else:
            return None
    
    def empty(self):
        return len(self.Q) == 0

In [49]:
def dijkstra(graph):
    Q = Priority_Queue()
    Q.put([0, graph.source])
    cur_costs = [[float('inf') for col in range(graph.width)] for row in range(graph.height)]
    prev = [[' ' for col in range(graph.width)] for row in range(graph.height)]
    cur_costs[graph.source[0]][graph.source[1]] = 0
    steps = 0
    
    while not Q.empty():
        steps += 1
        # Get the closest node
        closest = Q.get()
        current = closest[1]
        
        if current == graph.target:
            return cur_costs, prev, steps

        for i in range(len(directions)):
            n_row = current[0] + directions[i][0]
            n_col = current[1] + directions[i][1]
            if graph.in_bounds([n_row, n_col]):
                new_cost = cur_costs[current[0]][current[1]] + graph.cost([n_row, n_col])
                if new_cost < cur_costs[n_row][n_col]:
                    cur_costs[n_row][n_col] = new_cost
                    Q.put([new_cost, [n_row, n_col]])
                    prev[n_row][n_col] = i
                    
    return cur_costs, prev, steps

def build_path(prev, source, target):
    path = [[' ' for col in range(len(prev[row]))] for row in range(len(prev))]
    row = target[0]
    col = target[1]
    path[row][col] = '*'
    while row != source[0] or col != source[1]:
        new_row = row - directions[prev[row][col]][0]
        new_col = col - directions[prev[row][col]][1]
        path[new_row][new_col] = direction_symbols[prev[row][col]]
        row = new_row
        col = new_col
    return path

In [50]:
def run_dijkstra(map_select):
    graph = Graph(maps[map_select - 1])
    costs, prev, steps = dijkstra(graph)
    print_grid("Costs:", costs)
    path = build_path(prev, graph.source, graph.target)
    print_grid("Path", path),
    print("Number of steps: " + str(steps))
    display(Image(filename='./images/' + map_files[map_select-1]))
    
interact(run_dijkstra, map_select=[1,2,3])

interactive(children=(Dropdown(description=u'map_select', options=(1, 2, 3), value=1), Output()), _dom_classes=('widget-interact',))

<function __main__.run_dijkstra>

In [29]:
def heuristic(a, b):
    row_1, col_1 = a
    row_2, col_2 = b
    return abs(row_1 - row_2) + abs(col_1 - col_2)

def a_star(graph):
    Q = Priority_Queue()
    Q.put([0, graph.source])
    cur_costs = [[float('inf') for col in range(graph.width)] for row in range(graph.height)]
    prev = [[' ' for col in range(graph.width)] for row in range(graph.height)]
    cur_costs[graph.source[0]][graph.source[1]] = 0
    steps = 0
    
    while not Q.empty():
        steps += 1
        # Get the closest node
        closest = Q.get()
        current = closest[1]
        
        if current == graph.target:
            return cur_costs, prev, steps

        for i in range(len(directions)):
            n_row = current[0] + directions[i][0]
            n_col = current[1] + directions[i][1]
            if graph.in_bounds([n_row, n_col]):
                new_cost = cur_costs[current[0]][current[1]] + graph.cost([n_row, n_col])
                if new_cost < cur_costs[n_row][n_col]:
                    cur_costs[n_row][n_col] = new_cost
                    Q.put([new_cost + heuristic(graph.target, [n_row, n_col]), [n_row, n_col]])
                    prev[n_row][n_col] = i
                    
    return cur_costs, prev, steps

In [30]:
def run_a_star(map_select):
    graph = Graph(maps[map_select - 1])
    costs, prev, steps = a_star(graph)
    print_grid("Costs:", costs)
    path = build_path(prev, graph.source, graph.target)
    print_grid("Path", path),
    print("Number of steps: " + str(steps))
    display(Image(filename='./images/' + map_files[map_select-1]))
    
interact(run_a_star, map_select=[1,2,3])

interactive(children=(Dropdown(description=u'map_select', options=(1, 2, 3), value=1), Output()), _dom_classes=('widget-interact',))

<function __main__.run_a_star>

In [8]:
[0, 1, 2, 4, 6, 7, 8, 10]
[1, 2, 4, 7, 9, 9, 9, 10]
[3, 4, 7, 11, 12, 11, 12, 11]
[5, 7, 11, 14, 14, 14, 16, 14]
[7, 10, 13, 15, 15, 16, 19, 18]
[9, 11, 13, 14, 15, 16, 17, 20]
[10, 11, 12, 13, 16, 17, 18, 19]
[11, 15, 16, 17, 20, 20, 19, 20]

[0, 1, 2, 4, 6, 7, 8, 10]
[1, 2, 4, 7, 9, 9, 9, 10]
[3, 4, 7, 11, 12, 11, 12, 11]
[5, 7, 11, 14, 14, 14, 16, 14]
[7, 10, 13, 16, 16, 16, 20, 18]
[9, 11, 13, 14, 15, 16, 17, 20]
[10, 11, 12, 13, 16, 17, 18, 19]
[11, 15, 16, 17, 20, 20, 19, 20]

[11, 15, 16, 17, 20, 20, 19, 20]